In [ ]:
import torch
from molscribe import MolScribe
from rdkit import Chem
from rdkit.Chem import Draw
from huggingface_hub import hf_hub_download
#local_dir = 'Weights/'
#ckpt_path = hf_hub_download('yujieq/MolScribe', 'swin_base_char_aux_1m.pth', local_dir=local_dir)
model = MolScribe("Weights\swin_base_char_aux_1m.pth", device=torch.device('cpu'))
output = model.predict_image_file('Results/CN113402641A_page_2.png_0.png', return_atoms_bonds=True, return_confidence=True)
print("output",output[0]["chartok_coords"]["smiles"])
# mol = Chem.MolFromSmarts('CCCC[R]C(ccc)C[R]CC[R]c[R]c[R][R]C[SF]')
# img = Draw.MolToImage(mol)
# img.show()

In [3]:
from rdkit import Chem

# 创建一个包含虚拟原子的分子
smiles = "*[M](C)([X])C.[2*]c1c([3*])c2c(c1[3H]c1c([8*])c([9*])c3c1C([13*])=C([12*])C([11*])=C3[10*])C([7*])=C([6*])C([5*])=C2[4*]"
mol = Chem.MolFromSmarts(smiles)

if mol is not None:
    pred_molblock = Chem.MolToMolBlock(mol)
    print(pred_molblock)  # 显示MolBlock格式的输出
else:
    print("无法从提供的SMILES创建分子")


无法从提供的SMILES创建分子


In [11]:
mol = Chem.MolFromSmarts('[R2]c1cc2c(c1[Si](C)(C)c1c([R1])cc3c1C=CC=C3[Ar])C=CC=C2[Ar].[Ar]')
img = Draw.MolToImage(mol)
img.show()

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor

# 定义含有占位符的SMILES字符串
smiles = "c1c(c(c(c(c1*)*)*)*)C(=O)O*[N+](=O)[O-]"

# 从SMILES创建分子对象
mol = Chem.MolFromSmiles(smiles)
rdDepictor.Compute2DCoords(mol)

# 手动标记占位符原子的位置
r_atoms = [6, 7, 8, 9]  # 假设这些位置是占位符

# 添加原子注释，将这些位置标记为R1, R2, R3, R4
for i, atom in enumerate(mol.GetAtoms()):
    if i in r_atoms:
        atom.SetProp('atomNote', f'R{i - 5 + 1}')  # 调整注释的编号

# 绘制化学结构图
drawer = rdMolDraw2D.MolDraw2DCairo(300, 300)
drawer.DrawMolecule(mol)
drawer.FinishDrawing()
img = drawer.GetDrawingText()

# 将图像保存到文件或显示
with open("mol_image.png", "wb") as f:
    f.write(img)

# 如果在本地运行，可以直接显示图像
from IPython.display import Image
Image("mol_image.png")


In [ ]:
import torch
from rxnscribe import MolDetect
from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt

ckpt_path = hf_hub_download("Ozymandias314/MolDetectCkpt", "best_hf.ckpt")
model = MolDetect(ckpt_path, device=torch.device('cpu'))

image_file = "Pictures/5.png"
predictions = model.predict_image_file(image_file)
print(predictions)
visualize_images = model.draw_bboxes(predictions, image_file = image_file)
#保存为图片
for i, img in enumerate(visualize_images):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.axis('off')
    ax.imshow(img)
    plt.show()
    print(predictions[i])

In [ ]:
import torch
from rxnscribe import MolDetect
from huggingface_hub import hf_hub_download
from PIL import Image
import matplotlib.pyplot as plt
import os

# 下载并加载模型
ckpt_path = hf_hub_download("Ozymandias314/MolDetectCkpt", "best_hf.ckpt")
model = MolDetect(ckpt_path, device=torch.device('cuda'))

# 图片文件夹路径
image_folder = "Pictures"
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')]

# 遍历文件夹中的所有图片
for image_file in image_files:
    # 预测
    predictions = model.predict_image_file(image_file)
    # 读取图片
    original_image = Image.open(image_file)

    # 根据预测结果截取图片
    for i, prediction in enumerate(predictions):
        if prediction['category'] == '[Mol]':  # 检查类别是否为 '[Mol]'
            # 提取归一化的边界框坐标
            bbox = prediction['bbox']
            width, height = original_image.size
            
            # 将归一化坐标转换为像素坐标
            xmin = int(bbox[0] * width)
            ymin = int(bbox[1] * height)
            xmax = int(bbox[2] * width)
            ymax = int(bbox[3] * height)
            
            # 截取子图
            cropped_image = original_image.crop((xmin, ymin, xmax, ymax))
            
            # 保存子图
            cropped_filename = f'{os.path.basename(image_file)}_{i}.png'
            cropped_image.save("Results/" + cropped_filename)
            print(f"Saved {cropped_filename} with bbox: {bbox}")
    # 如果没有找到符合条件的边界框，则打印提示信息
    if not any(pred['category'] == '[Mol]' for pred in predictions):
        print(f"No molecules with category '[Mol]' were found in {image_file}.")


In [1]:
import torch
from molscribe import MolScribe
from rdkit import Chem
from rdkit.Chem import Draw
from huggingface_hub import hf_hub_download
import os
import pandas as pd

#local_dir = 'Weights/'
#ckpt_path = hf_hub_download('yujieq/MolScribe', 'swin_base_char_aux_1m.pth', local_dir=local_dir)
model = MolScribe("Weights/swin_base_char_aux_1m.pth", device=torch.device('cuda'))

# 创建空的DataFrame
results_df = pd.DataFrame(columns=['Filename', 'Prediction'])


directory_path = 'D:\Research\Chem_Structure\Results'
# 使用列表推导式和 os.path.isfile 来统计文件数量
file_count = len([f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))])
# 输出文件夹中的文件数量
print(f"Total number of files: {file_count}")

number = 0
for filename in os.listdir('Results'):
    if filename.endswith('.png'):  # 确保只处理PNG文件
        file_path = os.path.join('Results', filename)
        # 使用model.predict_image_file进行预测
        output = model.predict_image_file(file_path, return_atoms_bonds=True, return_confidence=True)
        output = output[0]["chartok_coords"]["smiles"]
        print(f"Filename: {filename}, Prediction: {output}")
        number += 1
        if number % 10 == 0:
            print(f"已处理 {number} 个文件")
        # 将文件名和预测结果添加到DataFrame
        results_df = results_df.append({'Filename': filename, 'Prediction': output}, ignore_index=True)

# 保存结果到CSV文件
results_df.to_csv('prediction_results.csv', index=False)

print("所有结果已保存到 prediction_results.csv 文件中。")


c:\Users\11707\anaconda3\Lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Total number of files: 1209


c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Filename: CN103952793A_page_2.png_0.png, Prediction: [2H]=[2H]
Filename: CN104045747A_page_3.png_0.png, Prediction: [H][IH]


C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Filename': filename, 'Prediction': output}, ignore_index=True)
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: F

...

C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Filename': filename, 'Prediction': output}, ignore_index=True)
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: F

Filename: WO2021126523A1_page_7.png_0.png, Prediction: [CH2+].[P-]


C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Filename': filename, 'Prediction': output}, ignore_index=True)
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\11707\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Filename: WO2021126523A1_page_8.png_0.png, Prediction: [T1]1[L1][M]([X])([X2])[L2]1
所有结果已保存到 prediction_results.csv 文件中。


C:\Users\11707\AppData\Local\Temp\ipykernel_52012\1197972381.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Filename': filename, 'Prediction': output}, ignore_index=True)
